# OpenEO Connection to Local EODC Backend

In [1]:
import openeo
import time
import logging
logging.basicConfig(level=logging.INFO)

/home/berni/miniconda3/envs/noworkflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
# Define constants

# west, south, east, north
datasets = [
{"west": 10.288696, "south": 45.935871, "east": 12.189331, "north": 46.905246, "crs": "EPSG:4326", "begin": "2017-05-01", "end": "2017-05-31"}, # running example
{"west": 26.330109, "south": -16.023376, "east": 28.171692, "north": -15.253714, "crs": "EPSG:4326", "begin": "2006-03-30", "end": "2006-03-30"}, # http:// dx.doi.org/ 10.3390/ rs8050402 1
{"west": 26.830673, "south": -15.307366, "east": 27.052460, "north": -15.113227, "crs": "EPSG:4326", "begin": "2007-03-30", "end": "2007-03-30"}, # http:// dx.doi.org/ 10.3390/ rs8050402 2
{"west": 25.563812, "south": -14.429360, "east": 26.092529, "north": -13.980713, "crs": "EPSG:4326", "begin": "2006-03-29", "end": "2006-03-31"}, # http:// dx.doi.org/ 10.3390/ rs8050402 3
{"west": -2.449951, "south": 51.771239, "east": -2.239838, "north": 51.890901, "crs": "EPSG:4326", "begin": "2007-07-23", "end": "2007-07-23"}, # http:// dx.doi.org/ 10.1016/ j.jag.2014.12.001 1
{"west": -2.449951, "south": 51.771239, "east": -2.239838, "north": 51.890901, "crs": "EPSG:4326", "begin": "2005-08-22", "end": "2005-08-22"}, # http:// dx.doi.org/ 10.1016/ j.jag.2014.12.001 2
{"west": -2.449951, "south": 51.771239, "east": -2.239838, "north": 51.890901, "crs": "EPSG:4326", "begin": "2007-07-23", "end": "2007-07-24"}, # http:// dx.doi.org/ 10.1016/j.jag.2016.12.003 1
{"west": 16.506958, "south": 47.529257, "east": 17.188110, "north": 48.022998, "crs": "EPSG:4326", "begin": "2007-07-23", "end": "2007-07-24"}, # Big Data Infrastructures for Processing Sentinel Data, Wolfgang Wagner
{"west": 104.276733, "south": 8.423470, "east": 106.809082, "north": 11.156845, "crs": "EPSG:4326", "begin": "2007-01-01", "end": "2011-01-01"}, # THE USE OF SAR BACKSCATTER TIME SERIES FOR CHARACTERISING RICE PHENOLOGY, DUY NGUYEN
]

# Connection
LOCAL_EODC_DRIVER_URL = "http://openeo.local.127.0.0.1.nip.io"


In [3]:
# Connect with GEE backend
con = openeo.connect(LOCAL_EODC_DRIVER_URL)
# Reset Mockupstate and database
con.resetdb()
con

In [4]:
# Choose dataset
processes = con.get_processes()
pgA = processes.get_collection(name="s2a_prd_msil1c")
pgA = processes.filter_daterange(pgA, extent=["2017-05-01", "2017-05-31"])
pgA = processes.filter_bbox(pgA, west=10.288696, south=45.935871, east=12.189331, north=46.905246, crs="EPSG:4326")

# Choose processes
pgA = processes.ndvi(pgA, nir="B08", red="B04")
pgA = processes.min_time(pgA)
pgA.graph

{'process_id': 'min_time',
 'imagery': {'process_id': 'NDVI',
  'imagery': {'process_id': 'filter_bbox',
   'imagery': {'process_id': 'filter_daterange',
    'imagery': {'process_id': 'get_collection', 'name': 's2a_prd_msil1c'},
    'extent': ['2017-05-01', '2017-05-31']},
   'extent': {'west': 10.288696,
    'east': 12.189331,
    'north': 46.905246,
    'south': 45.935871,
    'crs': 'EPSG:4326'}},
  'red': 'B04',
  'nir': 'B08'}}

In [5]:
# Create job A out of the process graph A (pgA)
jobA = con.create_job(pgA.graph)
jobA_id = jobA.job_id
jobA_id

'jb-f022b568-b674-48d7-9f98-d9555e5eb6f3'

In [6]:
# Start job at the backend
jobA.start_job()

202

In [7]:
# Wait until the job execution was finished, print job description
desc = jobA.describe_job
while desc["status"] == "submitted":
    desc = jobA.describe_job
desc["context_model"]

{'back_end_timestamp': '20190331181236.221436',
 'code_env': ['alembic==0.9.9',
  'amqp==1.4.9',
  'anyjson==0.3.3',
  'certifi==2018.8.24',
  'cffi==1.11.5',
  'chardet==3.0.4',
  'enum-compat==0.0.2',
  'eventlet==0.19.0',
  'gevent==1.3.6',
  'greenlet==0.4.14',
  'idna==2.7',
  'kombu==3.0.37',
  'Mako==1.0.7',
  'MarkupSafe==1.0',
  'marshmallow==2.15.3',
  'mock==2.0.0',
  'nameko==2.9.0',
  'nameko-sqlalchemy==1.4.0',
  'path.py==11.0.1',
  'pbr==4.0.4',
  'psycopg2==2.7.4',
  'pycparser==2.18',
  'pyOpenSSL==18.0.0',
  'python-dateutil==2.7.3',
  'python-editor==1.0.3',
  'PyYAML==3.12',
  'requests==2.20.0',
  'six==1.11.0',
  'SQLAlchemy==1.2.8',
  'urllib3==1.23',
  'Werkzeug==0.14.1',
  'wincertstore==0.2',
  'wrapt==1.10.11',
  'pytz==2018.7',
  'GitPython==2.1.11',
  'numpy==1.16.2',
  'GDAL==2.4.0'],
 'end_time': '2019-04-01 18:13:06.221436',
 'input_data': 'qu-3f0b66b8-3cb0-414d-bff4-95e08eb99393',
 'interpreter': 'Python 3.7.1',
 'job_id': 'jb-f022b568-b674-48d7-9f98-d

In [8]:
# retrieve input data pid of job A
pidA = jobA.get_data_pid_url()
pidA

'http://openeo.local.127.0.0.1.nip.io/collections/qu-3f0b66b8-3cb0-414d-bff4-95e08eb99393'

In [9]:
# Get current back end version and the version of the job Execution
versionA = jobA.get_backend_version()
version_old = con.version()
versionA

{'branch': 'master',
 'commit': '1a0cefd25c2a0fbb64a78cd9445c3c9314eaeb5b',
 'timestamp': '2019-03-31 18:12:36.221436',
 'url': 'https://github.com/bgoesswein/implementation_backend.git'}

In [11]:
# Versions should be the same.
(versionA["commit"] == version_old["commit"])

True

In [12]:
''' 2. Publish job Id and data PID  '''
(jobA.job_id + " using " + pidA)

'jb-f022b568-b674-48d7-9f98-d9555e5eb6f3 using http://openeo.local.127.0.0.1.nip.io/collections/qu-3f0b66b8-3cb0-414d-bff4-95e08eb99393'

In [13]:
''' 3. Update back end version  '''
# Switch to the Console of the Job Instance and update the repository and create a commit.

' 3. Update back end version  '

In [14]:
# Retrieve new back end version
version_new = con.version()
version_new

{'branch': 'master',
 'commit': '0f9a0e31fd768fe0a0e5009ea63e84483a0cc39c',
 'url': 'https://github.com/bgoesswein/implementation_backend.git'}

In [15]:
# The new Version should be different to the old one.
(version_new != version_old)

True

In [16]:
''' 4. Get old context by the job ID  '''
# Define Job with the job id.
jobA2 = con.get_job(jobA_id)
jobA2.job_id

'jb-f022b568-b674-48d7-9f98-d9555e5eb6f3'

In [17]:
# Get the version of the back end that was active during the job A execution
versionA2 = jobA2.get_backend_version()
versionA2

{'branch': 'master~1',
 'commit': '1a0cefd25c2a0fbb64a78cd9445c3c9314eaeb5b',
 'timestamp': '2019-03-31 18:12:36.221436',
 'url': 'https://github.com/bgoesswein/implementation_backend.git'}

In [18]:
# Old Version and job A version should be the same
(versionA2["commit"] == version_old["commit"] and (versionA2["commit"] != version_new["commit"]))

True